# Visualizando nuestras clasificaciónes

Para tener una mejor idea de las relaciones que existen entre nuestros datos, generaremos gráficas utilizando el softweare Yed y la paquetería pyyed.

In [1]:
import pandas as pd
import numpy as np

#pip install pyyed
import pyyed

## Función Generadora de Redes

In [2]:
# Crea un arrgelo con los códigos de identificación únicos
def get_array(df, col_1, col_2):
    
    list_cat = df[col_1].unique()
    
    array_list = []
    
    for style in list_cat:
        array_list.append(np.array((df[df[col_1] == style][col_2])))
    
    return array_list

#  Crea un diccionario del arreglo
def get_dic(df, col_1, cat):
    
    list_cat = df[col_1].unique()
    dic_list = []
    
    for i in range(len(cat)):
        dic_list.append({list_cat[i]:cat[i].tolist()})
    
    return dic_list, list_cat

# Todo junto
def get_list_dic(df, col_1, col_2):
    ls = get_array(df, col_1, col_2)
    
    return get_dic(df, col_1, ls)

In [3]:
# Genera los nodos de la red
def get_nodes(g, ls):
    for i in ls:
        g.add_node(i)

# Genera los nodos de la red con descrición
def get_nodes_wd(g, df, col_1, col_2):
    for i in range(len(df)):
        g.add_node(df[col_1][i], description = str(df[col_2][i]))
        
# Genera una lista de pares únicos sin repetición
def get_unique_pairs(ls):
    
    ls_1 = []
    
    for i in range(len(ls)):
        for j in ls[1+i:len(ls)]:
            ls_1.append((ls[i], j))
    return ls_1


# Genera conexiones entre pares únicos
def get_unique_pairs_edges(g, ls, color):
    
    for i in get_unique_pairs(ls):
        g.add_edge(i[0], i[1], color = color, arrowhead="none")

In [4]:
# Funcón generadora de Gráficas de Redes!!!

def fggr(df, col_1, col_2, string, color):
    
    list_final, idx = get_list_dic(df, col_1, col_2)
    df_1 = pd.DataFrame(idx).reset_index().rename(columns={0:"Category"})
    h = pyyed.Graph()
    
    for i in df_1['index']:
        get_nodes(h ,list_final[i][idx[i]])
        get_unique_pairs_edges(h, list_final[i][idx[i]], color)
        
        h.write_graph('graf/' + string + '.graphml')

## Nuestras Gráficas

In [6]:
Guidelines = pd.read_excel('SUP/2015_Guidelines.xlsx')
Guidelines.keys()[[1,2,3,4,5]]

Index(['BJCP Categories', 'Styles', 'Style Family', 'Style History', 'Origin'], dtype='object')

In [8]:
color = ['#0077BE','#AE0C00','#30BA8F','#FADA5E']
cat_var = ['BJCP Categories', 'Style Family', 'Style History', 'Origin']


#Este código genera una gráfica completa de todas las clasificaciones con una conección única por cada una de estas.
H = pyyed.Graph()

get_nodes(H, Guidelines['#'])

ls = []
idx = []

for i in cat_var:
    ls.append(get_list_dic(Guidelines, i, '#')[0])
    idx.append(get_list_dic(Guidelines, i, '#')[1])

for j in range(4):
    for k in range(len(ls[j])):
        get_unique_pairs_edges(H, ls[j][k][idx[j][k]], color[j])

H.write_graph('graf/complete.graphml')

#Este código genera gráficas individuales por categoría 
for i in range(len(cat_var)):
    fggr(Guidelines, cat_var[i], '#', cat_var[i], color[i])